In [32]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Relativer Pfad zur Datei
file_path = "titanic.csv"

# Datei laden
data = pd.read_csv(file_path)

# Unnötige Spalten entfernen
data_cleaned = data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1).copy()

# Fehlende Werte behandeln (ohne Kopie-Warnung)
data_cleaned.loc[:, 'Age'] = data_cleaned['Age'].fillna(data_cleaned['Age'].median())
data_cleaned.loc[:, 'Embarked'] = data_cleaned['Embarked'].fillna(data_cleaned['Embarked'].mode()[0])

# Kategorische Variablen umwandeln (One-Hot-Encoding)
categorical_features = ['Sex', 'Embarked']
numerical_features = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

X = data_cleaned.drop('Survived', axis=1)
y = data_cleaned['Survived']

# Daten vorverarbeiten
X_preprocessed = preprocessor.fit_transform(X)

# Train-/Test-Split
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, test_size=0.2, random_state=42)

# Aktivierungsfunktionen
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return sigmoid(x) * (1 - sigmoid(x))

# Initialisierung des neuronalen Netzes
def initialize_weights(input_size, hidden_size, output_size):
    np.random.seed(42)
    weights = {
        "W1": np.random.randn(input_size, hidden_size) * 0.01,
        "b1": np.zeros((1, hidden_size)),
        "W2": np.random.randn(hidden_size, output_size) * 0.01,
        "b2": np.zeros((1, output_size)),
    }
    return weights

# Vorwärtspropagation
def forward_propagation(X, weights):
    Z1 = np.dot(X, weights["W1"]) + weights["b1"]
    A1 = sigmoid(Z1)
    Z2 = np.dot(A1, weights["W2"]) + weights["b2"]
    A2 = sigmoid(Z2)
    cache = {"Z1": Z1, "A1": A1, "Z2": Z2, "A2": A2}
    return A2, cache

# Verlustfunktion (Binary Cross Entropy)
def compute_loss(y_true, y_pred):
    m = y_true.shape[0]
    loss = -(1 / m) * np.sum(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
    return loss

# Rückwärtspropagation
def backward_propagation(X, y, weights, cache):
    m = X.shape[0]
    A2 = cache["A2"]
    A1 = cache["A1"]

    dZ2 = A2 - y.reshape(-1, 1)
    dW2 = (1 / m) * np.dot(A1.T, dZ2)
    db2 = (1 / m) * np.sum(dZ2, axis=0, keepdims=True)

    dZ1 = np.dot(dZ2, weights["W2"].T) * sigmoid_derivative(cache["Z1"])
    dW1 = (1 / m) * np.dot(X.T, dZ1)
    db1 = (1 / m) * np.sum(dZ1, axis=0, keepdims=True)

    gradients = {"dW1": dW1, "db1": db1, "dW2": dW2, "db2": db2}
    return gradients

# Parameter-Update
def update_weights(weights, gradients, learning_rate):
    weights["W1"] -= learning_rate * gradients["dW1"]
    weights["b1"] -= learning_rate * gradients["db1"]
    weights["W2"] -= learning_rate * gradients["dW2"]
    weights["b2"] -= learning_rate * gradients["db2"]
    return weights

# Training des Netzwerks
def train_neural_network(X_train, y_train, input_size, hidden_size, output_size, epochs, learning_rate):
    weights = initialize_weights(input_size, hidden_size, output_size)

    for epoch in range(epochs):
        # Vorwärtspropagation
        y_pred, cache = forward_propagation(X_train, weights)

        # Verlust berechnen
        loss = compute_loss(y_train, y_pred)

        # Rückwärtspropagation
        gradients = backward_propagation(X_train, y_train, weights, cache)

        # Gewichte aktualisieren
        weights = update_weights(weights, gradients, learning_rate)

        # Fortschritt ausgeben
        if epoch % 100 == 0:
            print(f"Epoch {epoch}, Loss: {loss}")

    return weights

# Testen des Netzwerks
def predict(X, weights):
    y_pred, _ = forward_propagation(X, weights)
    return (y_pred > 0.5).astype(int)

# Hyperparameter und Training
input_size = X_train.shape[1]
hidden_size = 10
output_size = 1
epochs = 1000
learning_rate = 0.25

# Trainieren des Modells
weights = train_neural_network(X_train, y_train.to_numpy(), input_size, hidden_size, output_size, epochs, learning_rate)

# Vorhersage und Evaluierung
y_pred = predict(X_test, weights)
accuracy = np.mean(y_pred.flatten() == y_test.to_numpy())
print(f"Accuracy: {accuracy * 100:.2f}%")

#Zusammenfassung:
# In diesem Projekt haben wir ein neuronales Netz von Grund auf implementiert, um die Überlebenswahrscheinlichkeit von Titanic-Passagieren vorherzusagen. 
# Nach der Vorverarbeitung der Daten, bei der unnötige Spalten entfernt, fehlende Werte behandelt und die Daten standardisiert wurden, 
# haben wir ein einfaches Netzwerk mit einer versteckten Schicht und Sigmoid-Aktivierung erstellt. 
# Die Vorwärts- und Rückwärtspropagation sowie die Aktualisierung der Gewichte wurden manuell programmiert. 
# Der Trainingsprozess basierte auf Gradientenabstieg, und das Modell wurde anhand der Genauigkeit auf Testdaten bewertet.

# Verbesserungsmöglichkeiten:
# Der Code kann durch stabilere Verlustberechnungen, bessere Initialisierung der Gewichte und Regularisierungsmethoden wie L2 oder Dropout weiter verbessert werden, 
# um Überanpassung zu vermeiden. Zusätzlich könnten modernere Optimierungsalgorithmen wie Adam und Visualisierungen des Verlustverlaufs hinzugefügt werden, 
# um den Trainingsprozess transparenter zu gestalten. Abschließend wäre es hilfreich, weitere Metriken wie Precision und Recall zu integrieren, 
# um die Modellleistung genauer zu bewerten.

Epoch 0, Loss: 493.30588844178254
Epoch 100, Loss: 471.90726523143894
Epoch 200, Loss: 476.1174536427964
Epoch 300, Loss: 498.32426817718544
Epoch 400, Loss: 540.8366124945887
Epoch 500, Loss: 583.0799794522964
Epoch 600, Loss: 613.592397858941
Epoch 700, Loss: 632.5931566016475
Epoch 800, Loss: 643.6349845316676
Epoch 900, Loss: 649.9307315622932
Accuracy: 81.01%


In [33]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Relativer Pfad zur Datei
file_path = "C:\\Users\\marti\\Downloads\\Neuronales Netz\\titanic.csv"

# Datei laden
data = pd.read_csv(file_path)

# Unnötige Spalten entfernen
data_cleaned = data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1).copy()

# Fehlende Werte behandeln
data_cleaned.loc[:, 'Age'] = data_cleaned['Age'].fillna(data_cleaned['Age'].median())
data_cleaned.loc[:, 'Embarked'] = data_cleaned['Embarked'].fillna(data_cleaned['Embarked'].mode()[0])

# Kategorische Variablen umwandeln (One-Hot-Encoding)
categorical_features = ['Sex', 'Embarked']
numerical_features = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

X = data_cleaned.drop('Survived', axis=1)
y = data_cleaned['Survived']

# Daten vorverarbeiten
X_preprocessed = preprocessor.fit_transform(X)

# Aktivierungsfunktionen
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return sigmoid(x) * (1 - sigmoid(x))

# Initialisierung des neuronalen Netzes
def initialize_weights(input_size, hidden_size, output_size):
    np.random.seed(42)
    weights = {
        "W1": np.random.randn(input_size, hidden_size) * 0.01,
        "b1": np.zeros((1, hidden_size)),
        "W2": np.random.randn(hidden_size, output_size) * 0.01,
        "b2": np.zeros((1, output_size)),
    }
    return weights

# Vorwärtspropagation
def forward_propagation(X, weights):
    Z1 = np.dot(X, weights["W1"]) + weights["b1"]
    A1 = sigmoid(Z1)
    Z2 = np.dot(A1, weights["W2"]) + weights["b2"]
    A2 = sigmoid(Z2)
    cache = {"Z1": Z1, "A1": A1, "Z2": Z2, "A2": A2}
    return A2, cache

# Verlustfunktion (Binary Cross Entropy)
def compute_loss(y_true, y_pred):
    epsilon = 1e-8  # Kleine Konstante für Stabilität
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    m = y_true.shape[0]
    loss = -(1 / m) * np.sum(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
    return loss

# Rückwärtspropagation
def backward_propagation(X, y, weights, cache):
    m = X.shape[0]
    A2 = cache["A2"]
    A1 = cache["A1"]

    dZ2 = A2 - y.reshape(-1, 1)
    dW2 = (1 / m) * np.dot(A1.T, dZ2)
    db2 = (1 / m) * np.sum(dZ2, axis=0, keepdims=True)

    dZ1 = np.dot(dZ2, weights["W2"].T) * sigmoid_derivative(cache["Z1"])
    dW1 = (1 / m) * np.dot(X.T, dZ1)
    db1 = (1 / m) * np.sum(dZ1, axis=0, keepdims=True)

    gradients = {"dW1": dW1, "db1": db1, "dW2": dW2, "db2": db2}
    return gradients

# Parameter-Update
def update_weights(weights, gradients, learning_rate):
    weights["W1"] -= learning_rate * gradients["dW1"]
    weights["b1"] -= learning_rate * gradients["db1"]
    weights["W2"] -= learning_rate * gradients["dW2"]
    weights["b2"] -= learning_rate * gradients["db2"]
    return weights

# Training des Netzwerks
def train_neural_network(X_train, y_train, input_size, hidden_size, output_size, epochs, learning_rate):
    weights = initialize_weights(input_size, hidden_size, output_size)

    for epoch in range(epochs):
        # Vorwärtspropagation
        y_pred, cache = forward_propagation(X_train, weights)

        # Verlust berechnen
        loss = compute_loss(y_train, y_pred)

        # Rückwärtspropagation
        gradients = backward_propagation(X_train, y_train, weights, cache)

        # Gewichte aktualisieren
        weights = update_weights(weights, gradients, learning_rate)

        # Fortschritt ausgeben
        if epoch % 100 == 0:
            print(f"Epoch {epoch}, Loss: {loss}")

    return weights

# Testen des Netzwerks
def predict(X, weights):
    y_pred, _ = forward_propagation(X, weights)
    return (y_pred > 0.5).astype(int)

# Hyperparameter
input_size = X_preprocessed.shape[1]
hidden_size = 10
output_size = 1
epochs = 1000
learning_rate = 0.25

# Cross-Validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_accuracies = []

for fold, (train_index, test_index) in enumerate(skf.split(X_preprocessed, y)):
    print(f"\nFold {fold + 1}")
    X_train, X_test = X_preprocessed[train_index], X_preprocessed[test_index]
    y_train, y_test = y.to_numpy()[train_index], y.to_numpy()[test_index]

    # Trainieren des Modells
    weights = train_neural_network(X_train, y_train, input_size, hidden_size, output_size, epochs, learning_rate)

    # Vorhersage und Evaluierung
    y_pred = predict(X_test, weights)
    accuracy = np.mean(y_pred.flatten() == y_test)
    cv_accuracies.append(accuracy)
    print(f"Accuracy for Fold {fold + 1}: {accuracy * 100:.2f}%")

# Durchschnittliche Genauigkeit
mean_accuracy = np.mean(cv_accuracies)
print(f"\nCross-Validation Accuracy: {mean_accuracy * 100:.2f}%")



Fold 1
Epoch 0, Loss: 493.3182564142499
Epoch 100, Loss: 474.3337215002331
Epoch 200, Loss: 478.5074699906682
Epoch 300, Loss: 501.90913630407385
Epoch 400, Loss: 546.7750293315195
Epoch 500, Loss: 591.010039744103
Epoch 600, Loss: 622.7424733658241
Epoch 700, Loss: 642.3946529324004
Epoch 800, Loss: 653.8160877763995
Epoch 900, Loss: 660.4325295162313
Accuracy for Fold 1: 78.21%

Fold 2
Epoch 0, Loss: 494.01240853553196
Epoch 100, Loss: 475.2943125962951
Epoch 200, Loss: 479.64029289598614
Epoch 300, Loss: 504.58848544395244
Epoch 400, Loss: 552.1219372479734
Epoch 500, Loss: 598.2715969063767
Epoch 600, Loss: 630.6530705848091
Epoch 700, Loss: 650.1197099385706
Epoch 800, Loss: 661.1294131037565
Epoch 900, Loss: 667.3137906935644
Accuracy for Fold 2: 80.34%

Fold 3
Epoch 0, Loss: 494.01241115044763
Epoch 100, Loss: 475.35133772759224
Epoch 200, Loss: 480.63533512556285
Epoch 300, Loss: 509.6960338532325
Epoch 400, Loss: 561.7895615700019
Epoch 500, Loss: 610.3503698996842
Epoch 600,